In [1]:
# run this in your terminal / Anaconda prompt
!pip install pandas numpy rapidfuzz openpyxl xlrd matplotlib seaborn python-Levenshtein


  Using cached rapidfuzz-3.14.1-cp313-cp313-win_amd64.whl.metadata (12 kB)
Using cached rapidfuzz-3.14.1-cp313-cp313-win_amd64.whl (1.5 MB)

   ---------- ----------------------------- 1/4 [rapidfuzz]
   ---------------------------------------- 4/4 [python-Levenshtein]



In [1]:
# 01_data_preprocessing.ipynb - first cell
import os
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
from rapidfuzz import process, fuzz

# Base paths - change if your structure differs
BASE = Path(r"D:\Complete_Data\nutrition_project")
DATASETS = BASE / "Datasets"
RECIPE_DIR = DATASETS / "recipe_dataset"
NUTR_DIR = DATASETS / "nutrition_dataset"  # folder where USDA csv/xlsx files are
ENV_DIR = DATASETS / "environment_dataset"
COST_DIR = DATASETS / "cost_dataset"
PROCESSED = BASE / "processed_data"

# Make sure output folder exists
PROCESSED.mkdir(parents=True, exist_ok=True)

# Useful print to confirm
print("Paths set. Processed output will be saved to:", PROCESSED)


Paths set. Processed output will be saved to: D:\Complete_Data\nutrition_project\processed_data


In [5]:
# Load full_format_recipes.json
recipe_json_path = RECIPE_DIR / "full_format_recipes.json"
with open(recipe_json_path, "r", encoding="utf-8") as f:
    recipes_raw = json.load(f)

# Convert to DataFrame (top-level fields)
recipes_df = pd.json_normalize(recipes_raw)
print("Recipes rows:", len(recipes_df))
recipes_df.columns[:40]


Recipes rows: 20130


Index(['directions', 'fat', 'date', 'categories', 'calories', 'desc',
       'protein', 'rating', 'title', 'ingredients', 'sodium'],
      dtype='object')

In [6]:
# Create unique recipe_id (index) to track merges later
recipes_df = recipes_df.reset_index().rename(columns={'index':'recipe_id'})

# 1) ingredients.csv: one row per ingredient usage
ingredients_df = recipes_df[['recipe_id','title','ingredients']].explode('ingredients').reset_index(drop=True)
ingredients_df.rename(columns={'title':'recipe_title','ingredients':'ingredient_raw'}, inplace=True)

# 2) nutrition.csv: nutrition objects for each recipe (if present)
if 'nutrition' in recipes_df.columns:
    nutrition_df = pd.json_normalize(recipes_df['nutrition']).reset_index().rename(columns={'index':'recipe_id'})
    # nutrition_df may include keys like calories, protein... keep them
    # If nutrition is nested differently, inspect nutrition_df.head()
else:
    nutrition_df = pd.DataFrame(columns=['recipe_id'])
    
# Save the extracted files
ingredients_csv = RECIPE_DIR / "ingredients.csv"
nutrition_csv = RECIPE_DIR / "nutrition.csv"
ingredients_df.to_csv(ingredients_csv, index=False)
nutrition_df.to_csv(nutrition_csv, index=False)

print("Saved:", ingredients_csv, "rows:", len(ingredients_df))
print("Saved:", nutrition_csv, "rows:", len(nutrition_df))


Saved: D:\Complete_Data\nutrition_project\Datasets\recipe_dataset\ingredients.csv rows: 199106
Saved: D:\Complete_Data\nutrition_project\Datasets\recipe_dataset\nutrition.csv rows: 0


In [7]:
# Try .csv first; if not present, try .xlsx
def smart_read(path_no_ext):
    p_csv = Path(path_no_ext + ".csv")
    p_xlsx = Path(path_no_ext + ".xlsx")
    if p_csv.exists():
        return pd.read_csv(p_csv, low_memory=False)
    elif p_xlsx.exists():
        return pd.read_excel(p_xlsx)
    else:
        raise FileNotFoundError(f"Neither {p_csv} nor {p_xlsx} found.")

# Adjust to your exact folder name where you extracted USDA files
usda_folder = NUTR_DIR / "FoodData_Central_foundation_food_csv_2025-04-24"
food = smart_read(str(usda_folder / "usda_food"))
food_nutrient = smart_read(str(usda_folder / "usda_food_nutrient"))
nutrient_lookup = smart_read(str(usda_folder / "usda_nutrient_lookup"))
food_portion = None
try:
    food_portion = smart_read(str(usda_folder / "usda_food_portion"))
except FileNotFoundError:
    pass

print("USDA: food rows:", food.shape, "food_nutrient rows:", food_nutrient.shape, "nutrients:", nutrient_lookup.shape)


USDA: food rows: (74175, 5) food_nutrient rows: (155243, 11) nutrients: (477, 5)


In [8]:
# Show common nutrient names to find the exact text field
print("Sample nutrients:")
print(nutrient_lookup[['id','name','unit_name']].head(30))

# Find nutrient IDs for energy, protein, fat, carbs (case-insensitive search)
search_names = ['energy','protein','lipid','fat','carbohydrate','carbs','total lipid']
found = {}
for name in search_names:
    matches = nutrient_lookup[nutrient_lookup['name'].str.contains(name, case=False, na=False)]
    if not matches.empty:
        print(f"\nMatches for '{name}':")
        display(matches[['id','name','unit_name']].head(10))
        found[name] = matches

# Typical nutrient names: 'Energy', 'Protein', 'Total lipid (fat)', 'Carbohydrate, by difference'


Sample nutrients:
      id                                  name unit_name
0   2047      Energy (Atwater General Factors)      KCAL
1   2048     Energy (Atwater Specific Factors)      KCAL
2   1001                                Solids         G
3   1002                              Nitrogen         G
4   1003                               Protein         G
5   1004                     Total lipid (fat)         G
6   1005           Carbohydrate, by difference         G
7   1006  Fiber, crude (DO NOT USE - Archived)         G
8   1007                                   Ash         G
9   1008                                Energy      KCAL
10  1009                                Starch         G
11  1010                               Sucrose         G
12  1011                               Glucose         G
13  1012                              Fructose         G
14  1013                               Lactose         G
15  1014                               Maltose         G
16  1015     

id                               name unit_name
0   2047   Energy (Atwater General Factors)      KCAL
1   2048  Energy (Atwater Specific Factors)      KCAL
9   1008                             Energy      KCAL
63  1062                             Energy        kJ


Matches for 'protein':


id              name unit_name
4   1003           Protein         G
54  1053  Adjusted Protein         G


Matches for 'lipid':


...

id                                               name unit_name
5    1004                                  Total lipid (fat)         G
50   1049                                    Solids, non-fat         G
86   1085                                   Total fat (NLEA)         G
258  1257                           Fatty acids, total trans         G
259  1258                       Fatty acids, total saturated         G
291  1291  Fatty acids, other than 607-615, 617-621, 624-...         G
292  1292                 Fatty acids, total monounsaturated         G
293  1293                 Fatty acids, total polyunsaturated         G
318  1318                      Fatty acids, saturated, other         G
319  1319                     Fatty acids, monounsat., other         G


Matches for 'carbohydrate':


id                         name unit_name
6    1005  Carbohydrate, by difference         G
51   1050   Carbohydrate, by summation         G
73   1072          Carbohydrate, other         G
450  2039                Carbohydrates         G


Matches for 'total lipid':


id               name unit_name
5  1004  Total lipid (fat)         G

In [9]:
# Merge to add nutrient names to the mapping
fn = food_nutrient.merge(nutrient_lookup[['id','name','unit_name']], left_on='nutrient_id', right_on='id', how='left')

# Filter to nutrients we care about by name keywords
kw = ['energy','protein','total lipid','carbohydrate']
fn_filtered = fn[fn['name'].str.lower().str.contains('|'.join(kw), na=False)]

# pivot: rows = fdc_id, columns = nutrient name, values = amount
fn_pivot = (fn_filtered
            .pivot_table(index='fdc_id', columns=fn_filtered['name'], values='amount', aggfunc='mean')
            .reset_index())

# Merge pivot into food table
food_nutr = food.merge(fn_pivot, left_on='fdc_id', right_on='fdc_id', how='left')

# Rename common columns to tidy names (if present)
col_renames = {}
for col in food_nutr.columns:
    lc = col.lower()
    if 'energy' in lc:
        col_renames[col] = 'energy_kcal'
    elif 'protein' in lc and 'protein' not in col_renames.values():
        col_renames[col] = 'protein_g'
    elif 'total lipid' in lc or 'fat' in lc:
        col_renames[col] = 'fat_g'
    elif 'carbohydrate' in lc or 'carbs' in lc:
        col_renames[col] = 'carbs_g'
        
food_nutr.rename(columns=col_renames, inplace=True)

print("Food table with nutrients columns:", [c for c in food_nutr.columns if c in col_renames.values()])


Food table with nutrients columns: ['carbs_g', 'carbs_g', 'energy_kcal', 'energy_kcal', 'energy_kcal', 'protein_g', 'fat_g']


In [10]:
# Small helper: normalize text
def normalize_text(s):
    if pd.isna(s): return ""
    s = str(s).lower()
    s = re.sub(r'\([^)]*\)', '', s)          # remove parentheses
    s = re.sub(r'[^a-z0-9\s]', ' ', s)       # remove punctuation
    s = re.sub(r'\s+', ' ', s).strip()       # collapse spaces
    return s

# Clean ingredients from recipe ingredients.csv
ingredients_df['ingredient_clean'] = ingredients_df['ingredient_raw'].astype(str).apply(normalize_text)

# Clean USDA food descriptions
food_nutr['description_clean'] = food_nutr['description'].astype(str).apply(normalize_text)

# Clean cost file items - we'll read cost file below
print("Sample cleaned ingredient:", ingredients_df['ingredient_clean'].sample(5).tolist())


Sample cleaned ingredient: ['2 pound cultivated mussels scrubbed well and beards removed', '1 pound mussels cleaned', 'about 1 2 cup olive oil', '1 piece fresh or frozen turmeric peeled', '6 tablespoons olive oil']


In [11]:
# --- Environment Dataset ---
env_path = ENV_DIR / "Food_Production.csv"

# Auto-detect file type (CSV or Excel)
if env_path.suffix.lower() == ".csv":
    env_df = pd.read_csv(env_path)
else:
    env_df = pd.read_excel(env_path)

print("✅ Environment dataset loaded successfully!")
print("Shape:", env_df.shape)
print("Columns:", env_df.columns.tolist())

# Find a likely column name for the food item
possible_name_cols = [c for c in env_df.columns if 'food' in c.lower() or 'product' in c.lower() or 'item' in c.lower()]
env_name_col = possible_name_cols[0] if possible_name_cols else env_df.columns[0]

# Normalize names for merging
env_df['env_name_clean'] = env_df[env_name_col].astype(str).apply(normalize_text)



# --- Cost Dataset ---
cost_path_csv = COST_DIR / "wfp_food_prices_ind.csv"
cost_path_xlsx = COST_DIR / "wfp_food_prices_ind.xlsx"

# Use whichever exists
if cost_path_csv.exists():
    cost_df = pd.read_csv(cost_path_csv)
elif cost_path_xlsx.exists():
    cost_df = pd.read_excel(cost_path_xlsx)
else:
    raise FileNotFoundError("❌ No cost dataset found in COST_DIR!")

print("\n✅ Cost dataset loaded successfully!")
print("Shape:", cost_df.shape)
print("Columns:", cost_df.columns.tolist())

# Identify item/commodity/food name column
possible_cost_name_cols = [c for c in cost_df.columns if 'item' in c.lower() or 'commodity' in c.lower() or 'product' in c.lower()]
cost_name_col = possible_cost_name_cols[0] if possible_cost_name_cols else cost_df.columns[0]

# Normalize names for merging
cost_df['cost_name_clean'] = cost_df[cost_name_col].astype(str).apply(normalize_text)

# Display first few rows
print("\nSample rows from cost dataset:")
display(cost_df.head())


✅ Environment dataset loaded successfully!
Shape: (43, 23)
Columns: ['Food product', 'Land use change', 'Animal Feed', 'Farm', 'Processing', 'Transport', 'Packging', 'Retail', 'Total_emissions', 'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)', 'Eutrophying emissions per kilogram (gPO₄eq per kilogram)', 'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)', 'Freshwater withdrawals per 1000kcal (liters per 1000kcal)', 'Freshwater withdrawals per 100g protein (liters per 100g protein)', 'Freshwater withdrawals per kilogram (liters per kilogram)', 'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)', 'Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein)', 'Land use per 1000kcal (m² per 1000kcal)', 'Land use per kilogram (m² per kilogram)', 'Land use per 100g protein (m² per 100g protein)', 'Scarcity-weighted water use per kilogram (liters per kilogram)', 'Scarcity-weighted water use per 100g protein (liters per 100g protein)', 'Scarcit

C:\Users\Apoorva\AppData\Local\Temp\ipykernel_14888\3981267627.py:29: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  cost_df = pd.read_csv(cost_path_csv)



✅ Cost dataset loaded successfully!
Shape: (172018, 14)
Columns: ['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude', 'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency', 'price', 'usdprice']

Sample rows from cost dataset:


date      admin1      admin2            market   latitude  longitude  \
0       #date  #adm1+name  #adm2+name  #loc+market+name   #geo+lat   #geo+lon   
1  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
2  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
3  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   
4  1994-01-15       Delhi       Delhi             Delhi  28.666667  77.216667   

             category      commodity        unit         priceflag  \
0          #item+type     #item+name  #item+unit  #item+price+flag   
1  cereals and tubers           Rice          KG            actual   
2  cereals and tubers          Wheat          KG            actual   
3  miscellaneous food          Sugar          KG            actual   
4        oil and fats  Oil (mustard)          KG            actual   

          pricetype   currency   price    usdprice cost_name_clean  
0  #item+price+type  #currency  #value  #value+usd       item name  
1            Retail        INR     8.0      0.2545            rice  
2            Retail        INR     5.0       0.159           wheat  
3            Retail        INR    13.5      0.4294           sugar  
4            Retail        INR    31.0       0.986             oil

In [13]:
# Build candidate lists
usda_choices = food_nutr['description_clean'].dropna().unique().tolist()
cost_choices = cost_df['cost_name_clean'].dropna().unique().tolist()
env_choices = env_df['env_name_clean'].dropna().unique().tolist()

# Unique ingredient list to map (speeds up)
unique_ings = ingredients_df['ingredient_clean'].dropna().unique().tolist()

# Matching function with threshold - returns matched string from choices or None
def best_match(name, choices, score_cutoff=75):
    if not name:
        return None, 0
    res = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
    if res is None:
        return None, 0
    match, score, _ = res
    if score >= score_cutoff:
        return match, score
    return None, score

# Map ingredients -> usda, cost, env
map_rows = []
for ing in unique_ings:
    usda_match, usda_score = best_match(ing, usda_choices, score_cutoff=78)  # stricter for USDA
    cost_match, cost_score = best_match(ing, cost_choices, score_cutoff=75)
    env_match, env_score = best_match(ing, env_choices, score_cutoff=70)
    map_rows.append({'ingredient_clean':ing,
                     'usda_match':usda_match, 'usda_score':usda_score,
                     'cost_match':cost_match, 'cost_score':cost_score,
                     'env_match':env_match, 'env_score':env_score})
ing_map = pd.DataFrame(map_rows)

# Merge mapping back to ingredients_df
ingredients_mapped = ingredients_df.merge(ing_map, on='ingredient_clean', how='left')

# Save mapping for manual inspection and tuning later
ingredients_mapped.to_csv(PROCESSED / "ingredients_mapped.csv", index=False)
print("Saved ingredients mapping to:", PROCESSED / "ingredients_mapped.csv")


Saved ingredients mapping to: D:\Complete_Data\nutrition_project\processed_data\ingredients_mapped.csv


In [1]:
import numpy as np
from rapidfuzz import process, fuzz

print("🔹 Starting nutrient, cost, and environment merging (memory-safe)...")

# ----------------------------
# 1️⃣ Nutrient Mapping
# ----------------------------
nutrient_cols = [c for c in ['energy_kcal','protein_g','fat_g','carbs_g'] if c in food_nutr.columns]
nutrients_map = food_nutr[['description_clean','fdc_id'] + nutrient_cols].drop_duplicates(subset=['description_clean'])

# Build a dictionary for quick lookup
nutrient_lookup = nutrients_map.set_index('description_clean').to_dict(orient='index')

def get_nutrients(name):
    entry = nutrient_lookup.get(name)
    if entry:
        return entry
    return {'fdc_id': np.nan, 'energy_kcal': np.nan, 'protein_g': np.nan, 'fat_g': np.nan, 'carbs_g': np.nan}

# Apply in chunks
batch_size = 20000
nutrient_data = []
for start in range(0, len(ingredients_mapped), batch_size):
    end = min(start + batch_size, len(ingredients_mapped))
    batch = ingredients_mapped['usda_match'].iloc[start:end].apply(get_nutrients)
    nutrient_data.extend(batch)
    print(f"Processed nutrient batch {end}/{len(ingredients_mapped)}", end="\r")

# Convert back to DataFrame
nutrient_df = pd.DataFrame(nutrient_data)
ingredients_with_nutri = pd.concat([ingredients_mapped.reset_index(drop=True), nutrient_df], axis=1)
print("\n✅ Nutrient mapping done.")


# ----------------------------
# 2️⃣ Cost Mapping (Dictionary + Fuzzy)
# ----------------------------
price_col = None
for c in cost_df.columns:
    if 'price' in c.lower() or 'rs' in c.lower() or 'inr' in c.lower():
        price_col = c
        break

if price_col:
    cost_simple = cost_df[['cost_name_clean', price_col]].dropna().drop_duplicates()
    cost_lookup = dict(zip(cost_simple['cost_name_clean'], cost_simple[price_col]))
else:
    print("⚠️ No cost column found; setting NaN prices.")
    cost_lookup = {}

def get_cost(name):
    name = str(name).strip().lower()
    if name in cost_lookup:
        return cost_lookup[name]
    # fuzzy fallback
    match = process.extractOne(name, cost_lookup.keys(), scorer=fuzz.token_sort_ratio, score_cutoff=90)
    if match:
        return cost_lookup[match[0]]
    return np.nan

ingredients_with_nutri['price'] = ingredients_with_nutri['cost_match'].apply(get_cost)
print("✅ Cost mapping done.")


# ----------------------------
# 3️⃣ Environment Mapping
# ----------------------------
env_cols_use = [c for c in env_df.columns if any(k in c.lower() for k in ['co2','ghg','emission','water','land'])][:2]
env_simple = env_df[['env_name_clean'] + env_cols_use].dropna(subset=['env_name_clean']).drop_duplicates()
env_lookup = env_simple.set_index('env_name_clean').to_dict(orient='index')

def get_env_metrics(name):
    entry = env_lookup.get(name)
    if entry:
        return entry
    return {col: np.nan for col in env_cols_use}

env_data = []
for start in range(0, len(ingredients_with_nutri), batch_size):
    end = min(start + batch_size, len(ingredients_with_nutri))
    batch = ingredients_with_nutri['env_match'].iloc[start:end].apply(get_env_metrics)
    env_data.extend(batch)
    print(f"Processed environment batch {end}/{len(ingredients_with_nutri)}", end="\r")

env_df_expand = pd.DataFrame(env_data)
ingredients_with_nutri = pd.concat([ingredients_with_nutri.reset_index(drop=True), env_df_expand], axis=1)
print("\n✅ Environment mapping done.")


# ----------------------------
# 4️⃣ Save Final Dataset
# ----------------------------
output_path = PROCESSED / "ingredients_with_nutrients_and_cost.csv"
ingredients_with_nutri.to_csv(output_path, index=False)
print(f"\n💾 Saved enriched ingredient-level data to: {output_path}")
print("Shape:", ingredients_with_nutri.shape)


🔹 Starting nutrient, cost, and environment merging (memory-safe)...


NameError: name 'food_nutr' is not defined